# TODOs

1. Найти референсные последовательности для sbr - https://www.alliancegenome.org/gene/FB:FBgn0003321 (Sequence Details (Mode: cDNA)):
   - найти координаты гены на X хромосоме
   - полная последовательность гена
   - сплайсированный вариант
   - консервативная кассета
2. Написать срипт, который:
   - ищет ген sbr / nxf1 в базе данных NCBI для семейства Drosophilidae
   - выбирает из найденных только адекватные варианты и формирует из них БД для бласта
3. Сделать бласт вариантов из 1-го пункта на варианты из второго пункта

# Imports

In [1]:
import Bio.Blast
from Bio import Entrez
from Bio import Blast

from entrez import nucl_search, save_esearch_results
from parse_blast_results import extract_target_range, calculate_qc
from fasta_processing import plain_to_fasta, read_fasta

Entrez.email = "artemvaskaa@gmail.com"
Blast.email = "artemvaskaa@gmail.com"

# Fasta processing funcs

In [42]:
path_to_plain_dir = "/home/artemvaska/Master_degree/Diploma/References/"

In [ ]:
plain_to_fasta(path_to_plain_dir + "sbr_RA_gene_plain.fa", fasta_line_length=80, uppercase=True)

In [45]:
plain_to_fasta(path_to_plain_dir + "sbr_RA_CDS_plain.fa", fasta_line_length=80, uppercase=True)

# Entrez funcs

In [8]:
query = "(Drosophilidae[ORGN] NOT Drosophila melanogaster[ORGN]) AND (chromosome X[WORD] NOT PREDICTED[WORD] NOT gene[WORD]) AND 15000000:75000000[SLEN]"

In [9]:
id_list_test = nucl_search(query)

In [63]:
save_esearch_results(id_list_test[:3], "Drosophilidae")

In [13]:
# save all seqs in 1 file

with open("3_species.fa", "w") as ouf:
    for rec in id_list_test[:3]:
        lne = Entrez.efetch(
        db="nucleotide", id=rec, retmode="text", rettype="fasta"
        ).read()
        ouf.write(lne + "\n")

# Local blast+

Available DB in NCBI blastn -remote:

https://rc.dartmouth.edu/index.php/blast-introduction/blast-databases/

Установка blast+ на linux (https://www.ncbi.nlm.nih.gov/books/NBK52640/):

1. https://ftp.ncbi.nlm.nih.gov/blast/executables/LATEST/ (ncbi-blast-2.16.0+-x64-linux.tar.gz)
2. `mv ~/Downloads/ncbi-blast-2.16.0+-x64-linux.tar.gz ~`
3. `tar zxvpf ncbi-blast-2.16.0+-x64-linux.tar.gz`
4. `export PATH=$PATH:$HOME/ncbi-blast-2.16.0+/bin`
5. `mkdir $HOME/blastdb`
6. `export BLASTDB=$HOME/blastdb`
7. export into bash_profile (`nano .bash_profile`)

## Commands for local db

`$ makeblastdb -in Master_degree/Diploma/References/sbr_RA_gene.fa -dbtype nucl -out blastdb/sbr_ra/sbr_ra`

`$ blastn -db blastdb/sbr_ra -query Master_degree/Diploma/References/Drosophilidae/1797095071.fa`

`$ makeblastdb -in Master_degree/Diploma/References/3_species.fa -dbtype nucl -out blastdb/3_sp/3_sp`

`$ blastn -db blastdb/3_sp/3_sp -query Master_degree/Diploma/References/sbr_RA_gene.fa`

AE - NCBI - Genome project

AJ - EBI - Direct submissions

NM - curated RefSeq

# sbr gene

https://www.alliancegenome.org/gene/FB:FBgn0003321

Sequence Details (Mode: cDNA)

---

# Parsing BLAST XML2 and calculating QC

Drosophilidae (taxid:7214)

Drosophila melanogaster (taxid:7227)

https://biochem.slu.edu/bchm628/handouts/2013/Entrez_boolian_searches.pdf

https://biopython.org/docs/dev/Tutorial/chapter_blast.html

https://biopython.org/docs/dev/Tutorial/chapter_blast.html#the-blast-records-record-and-hit-classes

In [2]:
name_of_blast_res = "../Blast_res/full_sbr_RA_wgs_megablast_250_16.xml" # XML2 !!!
result_stream = open(name_of_blast_res, "rb")

In [3]:
blast_record = Blast.read(result_stream)

In [6]:
original_blast_record = blast_record[:]

In [7]:
hit = blast_record[0]
# hsp = hit[0] # HSPs -- “High-scoring Segment Pairs”
# hit.target  # hit.targets

In [8]:
print(hit)  # span -- the alignment length including gaps

Query: Query_719443
       sbr-RA-gene
  Hit: gi|2735068466|gb|JBBODO010000732.1| (length=1492563)
       Drosophila simulans strain SZ45 tig00001109, whole genome shotgun
       sequence
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0         0    9745.89    8606     [5873:14341]        [165809:174040]
          1         0    6167.08    4460      [1113:5494]        [161115:165507]
          2         0    1055.56     761       [360:1116]        [160331:161089]
          3   7.9e-81     315.05     231      [5500:5728]        [165482:165708]
          4     5e-43     189.48     217          [0:217]        [160127:160331]


In [9]:
hsp = hit[0]

In [10]:
print(hsp)

Query : Query_719443 Length: 14341 Strand: Plus
        sbr-RA-gene
Target: gi|2735068466|gb|JBBODO010000732.1| Length: 1492563 Strand: Plus
        Drosophila simulans strain SZ45 tig00001109, whole genome shotgun
        sequence

Score:9745 bits(5277), Expect:0,
Identities:7582/8606(88%),  Gaps:513.8606(6%)

gi|273506    165809 AGTTGAAAAGCAACTA-A-ATACCGCCGTCTAGAGTTCTAAATGCTTAATGGTAATTGGC
                  0 ||||||||||||||||-|-|||||-.|||||||||||.|||||.||||..|||||||||.
Query_719      5873 AGTTGAAAAGCAACTAGAGATACC-TCGTCTAGAGTTTTAAATTCTTATCGGTAATTGGG

gi|273506    165867 TTTACTTAATTACTCACTGCTACAATTACTTTGCCTGCCTTCAGGCGTTGCTAAAAAGC-
                 60 .|||.|||||||||||||||||||||||||||||||||||||||||||||||||||-||-
Query_719      5932 CTTATTTAATTACTCACTGCTACAATTACTTTGCCTGCCTTCAGGCGTTGCTAAAA-GCt

gi|273506    165926 -TTTTTCATAAACAAATTGC-ATAGCATAGTCTAAGTTGTAGTGCGCGACGAATCGGTGT
                120 -......|||||||||||||-||||||||||||||.|||||||||..|.|||||..||||
Query_719      5991 tttttttATAAACAAAT

In [289]:
# filter_func = lambda hit: len(hit) > 1
# len(blast_record) # 250
# blast_record[:] = filter(filter_func, blast_record)
# len(blast_record) # 228

# for hit in blast_record[:5]:  # quick check for the hit lengths
#     print(f"{hit.target.id} {len(hit)}")

In [11]:
extract_target_range(hit)

[160127, 174040]

In [12]:
qcs = calculate_qc(blast_record)

In [32]:
qcs_sorted = sorted(qcs.items(), key=lambda qc: qc[1], reverse=True)

In [24]:
QC_THRESHOLD = 0.5

In [169]:
for id, qc in qcs_sorted:
    if qc > QC_THRESHOLD:
        start, stop = extract_target_range(blast_record[id])
        break

print(start, stop)

160127 174040


In [33]:
def check_strands(hsp: Bio.Blast.HSP) -> (int, int):
    query_strand = (
        1 if hsp.coordinates[1][0] <= hsp.coordinates[1][-1] else 2
    )
    target_strand = (
        1 if hsp.coordinates[0][0] <= hsp.coordinates[0][-1] else 2
    )
    return query_strand, target_strand

In [34]:
for hsp in blast_record[qcs_sorted[0][0]]:
    print(check_strands(hsp))

(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)


In [167]:
blast_record[qcs_sorted[0][0]][0].target.seq

Seq({165809: 'AGTTGAAAAGCAACTAAATACCGCCGTCTAGAGTTCTAAATGCTTAATGGTAAT...TAG'}, length=1492563)

In [19]:
minus_hit = blast_record["gi|2733173904|gb|JBAMBW010000153.1|"]

In [20]:
print(minus_hit)

Query: Query_719443
       sbr-RA-gene
  Hit: gi|2733173904|gb|JBAMBW010000153.1| (length=10800764)
       Drosophila mauritiana strain Dmau_R32 contig_177, whole genome shotgun
       sequence
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0         0    6597.35    4971     [9403:14341]    [10651515:10646675]
          1         0    6270.50    4461      [1113:5494]    [10659647:10655239]
          2         0    3626.09    3585      [5873:9339]    [10654967:10651530]
          3         0    1050.02     761       [360:1116]    [10660426:10659673]
          4   7.9e-81     315.05     231      [5500:5728]    [10655264:10655038]
          5   1.4e-53     224.56     223          [0:223]    [10660638:10660420]


In [23]:
start, stop = extract_target_range(minus_hit)

In [27]:
# TODO

# seq = Entrez.efetch(db="nucleotide", id="JBAMBW010000153.1", strand=2, seq_start=start, seq_stop=stop, rettype="fasta")
# seq.read()